# Integrated Hybrid: Dynamic error correction
This sample builds from the first sample, _Integrated Hybrid: blending classical and quantum computing_, and creates a simple dynamic error correction scheme.

Find more about how to program with Integrated hybrid at https://aka.ms/AQ/Hybrid/Docs

In this program you will learn to:
1. **Represent a logical qubit** through a register.
2. **Perfom a dynamic error correction**. <br/>The error correction scheme is trivial. It assumes that a single bit-flip error has occurred and does not consider the possibility of errors occurring during the measurement of the ancillary qubits.

In [ ]:
%azure.connect "/subscriptions/subscriptionId/resourceGroups/resourceGroupName/providers/Microsoft.Quantum/Workspaces/workspaceName" location="westus"

In [ ]:
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

## 1. Encode a qubit register
The qubit register now represents a logical qubit.

In [ ]:
operation Encode(register : Qubit[]) : Unit is Adj {
    CNOT(register[0], register[1]);
    CNOT(register[0], register[2]);
}

## 2. Set value for each register
Arbitrarily, we set the register value to One.

In [ ]:
operation LogicalX(register : Qubit[]) : Unit {
    for qubit in register {
        X(qubit);
    }
}

## 3. Perform arbitrary operation
The Rx gate has a period of 4 * pi. Applying ```PI() * 0.5``` 8 times should return the qubit to its original state.
<br/>As we are manipulating physical qubits, we are however expecting some errors to be introduced.

In [ ]:
operation ApplyLogicalOperation(register : Qubit[]) : Unit is Adj {
    let theta = PI() * 0.5;
    for i in 1 .. 8 {
        for qubit in register {
            Rx(theta, qubit);
        }
    }
}

## 4. Verify the parity between qubits

In [ ]:
operation MeasureSyndrome(register : Qubit[]) : (Result, Result) {
    let parity01 = Measure([PauliZ, PauliZ, PauliI], register);
    let parity12 = Measure([PauliI, PauliZ, PauliZ], register);
    return (parity01, parity12);
}

## 5. Attempt to correct errors
You are leveraging a specific integrated hybrid capability here: branching based on measurement.

In [ ]:
operation CorrectError(register : Qubit[], parity01 : Result, parity12 : Result) : Bool {
    if (parity01 == One and parity12 == Zero) {
         X(register[0]); 
    }
    elif (parity01 == One and parity12 == One) {
        X(register[1]);
    }
    elif (parity01 == Zero and parity12 == One) {
        X(register[2]);
    }

    return parity01 == One or parity12 == One;
}

## 6. Define the DynamicBitFlipCode operation
Take all of the operations defined above and iterate multiple times:
1. Apply a logical operation that may introduce errors.
2. Attempt to correct introduced error and increment error count accordingly.
3. Measure the register to read the logical qubit value.
4. Report the value of each logical qubit.

In [ ]:
operation DynamicBitFlipCode() : (Result, Int){
    // Create a registers that represents a logical qubit.
    use localRegister = Qubit[3];

    // Apply several unitary operations to the encoded qubits performing error correction between each application.
    mutable corrections = 0;
    within {
        // Encode/Decode logical qubit.
        Encode(localRegister);
    }
    apply {
        LogicalX(localRegister);
        let iterations = 5;
        for _ in 1 .. iterations {
            // Apply unitary operations.
            ApplyLogicalOperation(localRegister);

            // Perform error correction and increase the counter if a correction was made.
            let (parityA01, parityA12) = MeasureSyndrome(localRegister);
            let correctedError = CorrectError(localRegister, parityA01, parityA12);
            if (correctedError) {
                set corrections += 1;
            }
        }
    }

    // Measure the first qubit in register and return value.
    let result = MResetZ(localRegister[0]);
    ResetAll(localRegister);
    return (result, corrections);
}

## 7. Submit the DynamicBitFlipCode operation
Using 50 shots, as configured below, **submitting this program will use approximately 11.31 units (HQCs or eHQCs)**.<br/>
If the job has not been completed after the client times out (300 seconds), you can query the status using `%azure.status` followed by querying the output using `%azure.output`.<br/>
If the kernel was restarted after the program had been submitted, you will have to use `%azure.status <job_id>` to get the status of the job and `%azure.output <job_id>` to retrieve the results.<br/>
You can also check the status of the job under the **Job management** section of the portal.

**Understanding the results**<br/>
The histogram reports the ratio of the various outputs.</br>
For instance, if you get:</br>
- `Result: (1,0) Frequency: 0.92` means that in 92% of the total number of shots the operation returned a qubit with the value One and that no error correction was necessary.</br>
- `Result: (0,2) Frequency: 0.02` means that in 2% of the total number of shots the operation returned a qubit with the value Zero and that 2 error corrections were performed. The error correction failed as the value flipped.</br>
- `Result: (0,0) Frequency: 0` means that in 2% of the total number of shots the operation returned a qubit with the value One and that 2 error corrections were performed. The error correction failed as the value flipped.</br>
- `Result: (1,2) Frequency: 0.02` means that in 2% of the total number of shots the operation returned a qubit with the value One and that 2 error corrections were performed.

In [ ]:
%azure.target quantinuum.sim.h1-1e-preview
%azure.target-capability AdaptiveExecution
%azure.execute DynamicBitFlipCode shots=50 timeout=300 poll=10

In [ ]:
%azure.status

In [ ]:
%azure.output

# ↗ Run an advanced experiment - Iterative phase estimation
Now that you have learned the basics of Integrated hybrid, experiment with the Iterative phase estimation sample available in the sample gallery as well.</br>
To learn more, please refer to the [online documentation](https://aka.ms/AQ/Hybrid/Docs).